In [1]:
import os
import json
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from skimage.morphology import skeletonize
from scipy.ndimage import distance_transform_edt

# ---------------- CONFIG ----------------
ROI_DIR = "roi"
MASK_DIR = "masks_improved"
META_DIR = "roi_meta"
OUT_DIR = "resultsss"
VIS_DIR = os.path.join(OUT_DIR, "visuals")
CSV_PATH = os.path.join(OUT_DIR, "stenosis_results.csv")

MIN_INSIDE = 1
MIN_OUTSIDE = 10
# ----------------------------------------

os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(VIS_DIR, exist_ok=True)

rows = []

roi_files = [f for f in os.listdir(ROI_DIR)
             if f.lower().endswith((".png", ".jpg", ".jpeg"))]

for roi_name in tqdm(roi_files):
    roi_path = os.path.join(ROI_DIR, roi_name)
    mask_path = os.path.join(
        MASK_DIR,
        os.path.splitext(roi_name)[0] + "_mask.png"
    )
    meta_path = os.path.join(
        META_DIR,
        os.path.splitext(roi_name)[0] + ".json"
    )

    if not (os.path.exists(mask_path) and os.path.exists(meta_path)):
        continue

    roi = cv2.imread(roi_path)
    mask = cv2.imread(mask_path, 0)

    if roi is None or mask is None:
        continue

    with open(meta_path) as f:
        meta = json.load(f)

    sx1, sy1, sx2, sy2 = meta["yolo_box"]

    binary = mask > 0
    skeleton = skeletonize(binary)

    if skeleton.sum() < 20:
        continue

    dist = distance_transform_edt(binary)
    coords = np.column_stack(np.where(skeleton))
    diameters = dist[skeleton] * 2

    inside = (
        (coords[:,1] >= sx1) & (coords[:,1] <= sx2) &
        (coords[:,0] >= sy1) & (coords[:,0] <= sy2)
    )

    outside = ~inside

    diam_inside = diameters[inside]
    diam_outside = diameters[outside]

    if len(diam_inside) < MIN_INSIDE or len(diam_outside) < MIN_OUTSIDE:
        continue

    D_min = np.min(diam_inside)
    D_ref = np.mean(np.sort(diam_outside)[-10:])
    stenosis = (1 - D_min / D_ref) * 100

    # ---------- SAVE VISUAL ----------
    vis = roi.copy()

    cv2.rectangle(vis, (sx1, sy1), (sx2, sy2), (255, 0, 0), 2)

    idx_min = np.where(inside)[0][np.argmin(diam_inside)]
    y_min, x_min = coords[idx_min]
    cv2.circle(vis, (x_min, y_min), 4, (0, 0, 255), -1)

    idx_ref = np.where(outside)[0][np.argmax(diam_outside)]
    y_ref, x_ref = coords[idx_ref]
    cv2.circle(vis, (x_ref, y_ref), 4, (0, 255, 0), -1)

    cv2.putText(
        vis,
        f"Stenosis: {stenosis:.1f}%",
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.8,
        (0, 255, 255),
        2
    )

    cv2.imwrite(
        os.path.join(VIS_DIR, roi_name),
        vis
    )

    # ---------- SAVE CSV ROW ----------
    rows.append({
        "image": roi_name,
        "D_min_px": round(float(D_min), 3),
        "D_ref_px": round(float(D_ref), 3),
        "stenosis_percent": round(float(stenosis), 2)
    })

# ---------- EXPORT CSV ----------
df = pd.DataFrame(rows)
df.to_csv(CSV_PATH, index=False)

print(f"✅ Processed {len(df)} images")
print(f"📄 CSV saved to {CSV_PATH}")
print(f"🖼️ Visuals saved to {VIS_DIR}")



100%|██████████| 1233/1233 [00:32<00:00, 37.62it/s]

✅ Processed 1007 images
📄 CSV saved to resultsss\stenosis_results.csv
🖼️ Visuals saved to resultsss\visuals
